In [60]:
import pandas as pd
import pyodbc
import os
from unidecode import unidecode
from sqlalchemy import create_engine

In [61]:
# Conexão com banco de dados
conexao = pyodbc.connect("DRIVER={SQL Server};SERVER=DESKTOP-51H10JN;DATABASE=DW-VendaCombustivel;Trusted_Connection=yes")

In [62]:
# Função para ler cada csv e inserir ele em um DF
def LerArquivosVendas(caminho, nome_arquivo):

    arquivo = f'{caminho}{nome_arquivo}'

    colunas = [
        'COMBUSTIVEL',
        'ANO',
        'REGIAO',
        'ESTADO',
        'UNIDADE',
        'JANEIRO',
        'FEVEREIRO',
        'MARCO',
        'ABRIL',
        'MAIO',
        'JUNHO',
        'JULHO',
        'AGOSTO',
        'SETEMBRO',
        'OUTUBRO',
        'NOVEMBRO',
        'DEZEMBRO',
        'TOTAL'
    ]

    df = pd.read_csv(arquivo, sep=';', names=colunas, header=None, skiprows=1, decimal=',', index_col=False)

    df['CAMINHO'] = caminho
    df['ARQUIVO'] = nome_arquivo

    return df


In [63]:
# Função para concatenar todos os arquivos presentes no caminho especificado
def ConcatenaArquivosVendas(caminho):

    # Cria um array contendo o nome de todos os arquivos
    nomes_arquivos = os.listdir(caminho)
    
    # Passa por cada arquivo, chama a função que faz a leitura deles, le e concatena
    for i, y in enumerate(nomes_arquivos):

        df = LerArquivosVendas(caminho, y)
        if i == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])
    
    return df_final

In [64]:
caminho = 'C:\\Users\\Rhey\\Downloads\\DW-VendasCombustivel_v2\\Arquivos\\Vendas\\'

df_final = ConcatenaArquivosVendas(caminho)

C:\Users\Rhey\AppData\Local\Temp\ipykernel_12180\2048166930.py:27: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(arquivo, sep=';', names=colunas, header=None, skiprows=1, decimal=',', index_col=False)
C:\Users\Rhey\AppData\Local\Temp\ipykernel_12180\2048166930.py:27: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(arquivo, sep=';', names=colunas, header=None, skiprows=1, decimal=',', index_col=False)


In [65]:
# Para avaliar todas as colunas disponíveis
df_final.columns

Index(['COMBUSTIVEL', 'ANO', 'REGIAO', 'ESTADO', 'UNIDADE', 'JANEIRO',
       'FEVEREIRO', 'MARCO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO',
       'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO', 'TOTAL', 'CAMINHO',
       'ARQUIVO'],
      dtype='object')

In [66]:
# retira espaços, acentuação e padroniza tudo como maiusculo
df_final['COMBUSTIVEL'] = df_final['COMBUSTIVEL'].astype(str).str.strip().str.upper().apply(unidecode)
df_final['REGIAO'] = df_final['REGIAO'].astype(str).str.strip().str.upper().apply(unidecode)
df_final['ESTADO'] = df_final['ESTADO'].astype(str).str.strip().str.upper().apply(unidecode)
df_final['UNIDADE'] = df_final['UNIDADE'].astype(str).str.strip().str.upper().apply(unidecode)

# Concatenar as colunas 'ESTADO' e 'REGIAO' em uma nova coluna 'localidade'
df_final['LOCALIDADE'] = df_final.apply(lambda row: f"{row['REGIAO']} - {row['ESTADO']}", axis=1)

In [67]:
print(df_final['COMBUSTIVEL'].unique())
print(df_final['ESTADO'].unique())

['ETANOL HIDRATADO' 'GASOLINA C' 'OLEO DIESEL']
['RONDONIA' 'ACRE' 'AMAZONAS' 'RORAIMA' 'PARA' 'AMAPA' 'TOCANTINS'
 'MARANHAO' 'PIAUI' 'CEARA' 'RIO GRANDE DO NORTE' 'PARAIBA' 'PERNAMBUCO'
 'ALAGOAS' 'SERGIPE' 'BAHIA' 'MINAS GERAIS' 'ESPIRITO SANTO'
 'RIO DE JANEIRO' 'SAO PAULO' 'PARANA' 'SANTA CATARINA'
 'RIO GRANDE DO SUL' 'MATO GROSSO DO SUL' 'MATO GROSSO' 'GOIAS'
 'DISTRITO FEDERAL']


In [68]:
# Transformações coluna Combustivel e criação da tabela dimensao

df_combustivel = pd.DataFrame({
    'combustivel': df_final['COMBUSTIVEL']
    })
    
# retira duplicatas
df_combustivel = df_combustivel.drop_duplicates()
# reseta o index (para o df esquecer do index do arquivo original)
df_combustivel = df_combustivel.reset_index(drop=True)

df_combustivel['id'] = df_combustivel.reset_index().index

# Criar um dicionário de mapeamento entre os combustiveis e seus índices
mapeamento_combustivel = df_combustivel['combustivel'].reset_index().set_index('combustivel')['index'].to_dict()
# Aplicar o mapeamento à coluna 'ANO' em df_final
df_final['COMBUSTIVEL'] = df_final['COMBUSTIVEL'].map(mapeamento_combustivel)

In [69]:
# Transformações coluna Ano e criação da tabela dimensao
df_tempo = pd.DataFrame({
    'ano': df_final['ANO']
    })

# retira duplicatas
df_tempo = df_tempo.drop_duplicates()

# reseta o index (para o df esquecer do index do arquivo original)
df_tempo = df_tempo.reset_index(drop=True)

df_tempo['id'] = df_tempo.reset_index().index

# Criar um dicionário de mapeamento entre os anos e seus índices
mapeamento_anos = df_tempo['ano'].reset_index().set_index('ano')['index'].to_dict()
# Aplicar o mapeamento à coluna 'ANO' em df_final
df_final['ANO'] = df_final['ANO'].map(mapeamento_anos)

In [70]:
# Transformações coluna Estado e Região e criação da tabela dimensao
df_localidade = pd.DataFrame({
    'regiao': df_final['REGIAO'],
    'estado': df_final['ESTADO']
    })

# retira duplicatas
df_localidade = df_localidade.drop_duplicates()

# reseta o index (para o df esquecer do index do arquivo original)
df_localidade = df_localidade.reset_index(drop=True)

df_localidade['localidade'] = df_localidade.apply(lambda row: f"{row['regiao']} - {row['estado']}", axis=1)

df_localidade['id'] = df_localidade.reset_index().index

# Criar um dicionário de mapeamento entre os combustiveis e seus índices
mapeamento_localidade = df_localidade['localidade'].reset_index().set_index('localidade')['index'].to_dict()
# Aplicar o mapeamento à coluna 'ANO' em df_final
df_final['LOCALIDADE'] = df_final['LOCALIDADE'].map(mapeamento_localidade)

In [71]:
# Excluir as colunas 'ESTADO' e 'REGIAO'
df_final.drop(['ESTADO', 'REGIAO', 'UNIDADE'], axis=1, inplace=True)

In [72]:
# Convertendo de m3 para litros

df_final['JANEIRO'] = df_final['JANEIRO'] * 1000
df_final['FEVEREIRO'] = df_final['FEVEREIRO'] * 1000
df_final['MARCO'] = df_final['MARCO'] * 1000
df_final['ABRIL'] = df_final['ABRIL'] * 1000
df_final['MAIO'] = df_final['MAIO'] * 1000
df_final['JUNHO'] = df_final['JUNHO'] * 1000
df_final['JULHO'] = df_final['JULHO'] * 1000
df_final['AGOSTO'] = df_final['AGOSTO'] * 1000
df_final['SETEMBRO'] = df_final['SETEMBRO'] * 1000
df_final['OUTUBRO'] = df_final['OUTUBRO'] * 1000
df_final['NOVEMBRO'] = df_final['NOVEMBRO'] * 1000
df_final['DEZEMBRO'] = df_final['DEZEMBRO'] * 1000
df_final['TOTAL'] = df_final['TOTAL'] * 1000

In [73]:
#Transfere todos os dataframes para uma tabela em SQL Server
connection_string = f"mssql+pyodbc://DESKTOP-51H10JN/DW-VendaCombustivel?driver=ODBC Driver 17 for SQL Server"
engine = create_engine(connection_string, fast_executemany=True)

df_combustivel.to_sql(name='DIM_COMBUSTIVEL', con=engine, index=False, if_exists='replace')
df_localidade.to_sql(name='DIM_LOCALIDADE', con=engine, index=False, if_exists='replace')
df_tempo.to_sql(name='DIM_TEMPO', con=engine, index=False, if_exists='replace')
df_final.to_sql(name='FATO_VENDAS_COMBUSTIVEL', con=engine, index=True, if_exists='replace')



-1

In [74]:
-- Comandos SQL

select *
from DIM_COMBUSTIVEL

SELECT *
FROM DIM_LOCALIDADE

SELECT *
FROM DIM_TEMPO

SELECT *
FROM FATO_VENDAS_COMBUSTIVEL

UPDATE DIM_COMBUSTIVEL
SET combustivel = 'GASOLINA COMUM'
where combustivel = 'GASOLINA C'

SyntaxError: invalid syntax (48908869.py, line 1)

In [ ]:
df_combustivel

,combustivel
0,ETANOL HIDRATADO
1,GASOLINA DE AVIACAO
2,GASOLINA C
3,OLEO DIESEL
4,QUEROSENE DE AVIACAO
